In [1]:
import pandas as pd 
import ast
import json
from pandas import json_normalize

.
## ==================================================
# Boliga_propertySales_additionalFloorInfos.csv
## ==================================================
.

## ----- read in the file : 

In [2]:
path = r'D:\Thesis\Properties\Denmark\RE_due_scraping_properties\Boliga_dk\Creating_main_dataset_for_sales_data\Data_split\11_salesInfos\Boliga_propertySales_salesInfos.csv'
data_salesInfo = pd.read_csv(path, encoding='utf-8',low_memory=False)
data_salesInfo.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1031507 entries, 0 to 1031506
Data columns (total 17 columns):
 #   Column                                  Non-Null Count    Dtype 
---  ------                                  --------------    ----- 
 0   RowID_MAIN_boliga_ROW_ID_unitID         1031507 non-null  object
 1   RowID_propertyCode_bfeCode              1031507 non-null  int64 
 2   RowID_municipalityCode                  1031507 non-null  int64 
 3   RowID_propertyCode_esrCode              1031507 non-null  int64 
 4   Address_info_addressString              1031507 non-null  object
 5   Address_info_streetString               1031507 non-null  object
 6   Address_info_houseNumber                1031507 non-null  object
 7   Address_info_floorString                270077 non-null   object
 8   Address_info_sideDoorNumber             215286 non-null   object
 9   Address_info_zipString_code_and_name    1031507 non-null  object
 10  Address_info_zipCode                    10

### --------- Convert "RAW_dictionary_salesInfos" column to dictionary object                

In [3]:
def decode_json(x):
    try:
        if isinstance(x, str) and x.strip() != "" and not pd.isna(x):
            return ast.literal_eval(x)
            # Alternatively, you can use json.loads as follows:
            # return json.loads(x.replace("'", '"'))
        else:
            return x
    except (ValueError, SyntaxError) as e:
        print(f"Error decoding JSON: {e}")
        return None

# Apply the decoding function to the specified column
data_salesInfo['RAW_dictionary_salesInfos'] = data_salesInfo['RAW_dictionary_salesInfos'].apply(decode_json)

In [6]:
data_salesInfo['RAW_dictionary_salesInfos'][0]

[{'isSalesValid': True,
  'handoverCode': '1',
  'handoverName': 'Alm. frit salg',
  'deedIssueDate': '2007-09-12T00:00:00',
  'price': 14100000,
  'recalculationDate': '2006-11-17T00:00:00',
  'rebuildYear': 2008},
 {'isSalesValid': True,
  'handoverCode': '1',
  'handoverName': 'Alm. frit salg',
  'deedIssueDate': '2023-01-18T00:00:00',
  'price': 3050000,
  'recalculationDate': '2023-11-08T00:00:00',
  'rebuildYear': 2008},
 {'isSalesValid': True,
  'handoverCode': '1',
  'handoverName': 'Alm. frit salg',
  'deedIssueDate': '2017-12-11T00:00:00',
  'price': 2910000,
  'recalculationDate': '2017-08-30T00:00:00',
  'rebuildYear': 2008},
 {'isSalesValid': True,
  'handoverCode': '1',
  'handoverName': 'Alm. frit salg',
  'deedIssueDate': '2007-09-27T00:00:00',
  'price': 2600000,
  'recalculationDate': '2007-06-14T00:00:00',
  'rebuildYear': 2008}]

## --- Count number of TOTAL SALES

In [8]:
salescounter = 0 

for list_item in data_salesInfo['RAW_dictionary_salesInfos']:

    for salesrecord in list_item:
        salescounter+=1

salescounter

2623340

In [10]:
2623340/1031507 

2.5432110494645213

# ------- Count number of sales by sales type

In [12]:
sales_category = [] 

for list_item in data_salesInfo['RAW_dictionary_salesInfos']:

    if list_item !='' and list_item !=[] and isinstance(list_item,list):
        for salesrecord in list_item:
            sales_category.append(salesrecord['handoverName'])



from collections import Counter

# Use Counter to count the frequency of each item in the list
item_frequency = Counter(sales_category)

# Display the frequency of each item
for item, count in item_frequency.items():
    print(f"{item}: {count} times")

Alm. frit salg: 2404797 times
Familieoverdragelse: 120039 times
Salg i øvrigt: 57792 times
Auktion: 40712 times


# ------ Explode the columns : RAW_dictionary_salesInfos

In [5]:
data_salesInfo = data_salesInfo.explode('RAW_dictionary_salesInfos').reset_index(drop=True)

In [6]:
data_salesInfo.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 2623800 entries, 0 to 2623799
Data columns (total 17 columns):
 #   Column                                  Dtype 
---  ------                                  ----- 
 0   RowID_MAIN_boliga_ROW_ID_unitID         object
 1   RowID_propertyCode_bfeCode              int64 
 2   RowID_municipalityCode                  int64 
 3   RowID_propertyCode_esrCode              int64 
 4   Address_info_addressString              object
 5   Address_info_streetString               object
 6   Address_info_houseNumber                object
 7   Address_info_floorString                object
 8   Address_info_sideDoorNumber             object
 9   Address_info_zipString_code_and_name    object
 10  Address_info_zipCode                    int64 
 11  Feature_dummy_isApartment               bool  
 12  Feature_categorical_propertyTypeNumber  int64 
 13  Feature_Floor_level_of_property         object
 14  Feature_category_propertyTypeName       object
 15

In [7]:
data_salesInfo = data_salesInfo.join(json_normalize(data_salesInfo.pop('RAW_dictionary_salesInfos')))
data_salesInfo.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 2623800 entries, 0 to 2623799
Data columns (total 23 columns):
 #   Column                                  Dtype  
---  ------                                  -----  
 0   RowID_MAIN_boliga_ROW_ID_unitID         object 
 1   RowID_propertyCode_bfeCode              int64  
 2   RowID_municipalityCode                  int64  
 3   RowID_propertyCode_esrCode              int64  
 4   Address_info_addressString              object 
 5   Address_info_streetString               object 
 6   Address_info_houseNumber                object 
 7   Address_info_floorString                object 
 8   Address_info_sideDoorNumber             object 
 9   Address_info_zipString_code_and_name    object 
 10  Address_info_zipCode                    int64  
 11  Feature_dummy_isApartment               bool   
 12  Feature_categorical_propertyTypeNumber  int64  
 13  Feature_Floor_level_of_property         object 
 14  Feature_category_propertyTypeName 

In [17]:
len(data_salesInfo[data_salesInfo['isSalesValid'].isna()])

460

In [20]:
data_salesInfo['isSalesValid'] = data_salesInfo['isSalesValid'].replace(0,False)
data_salesInfo['isSalesValid'].value_counts() 

isSalesValid
True     2623340
False        460
Name: count, dtype: int64

In [10]:
import numpy as np 
np.mean(data_salesInfo['price'])

2812616.955758308

In [11]:
np.min(data_salesInfo['price'])

1.0

In [12]:
np.max(data_salesInfo['price'])

1299000000.0

# ----- Rename columns:

In [22]:
change_column_names = {
    'isSalesValid':'salesInfo_isSalesValid', 
    'handoverCode':'salesInfo_handoverCode', 
    'handoverName':'salesInfo_handoverName', 
    'deedIssueDate':'salesInfo_deedIssueDate', 
    'price':'salesInfo_price',
    'recalculationDate':'salesInfo_recalculationDate', 
    'rebuildYear':'salesInfo_rebuildYear'    
}

data_salesInfo = data_salesInfo.rename(columns=change_column_names)
data_salesInfo.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 2623800 entries, 0 to 2623799
Data columns (total 23 columns):
 #   Column                                  Dtype  
---  ------                                  -----  
 0   RowID_MAIN_boliga_ROW_ID_unitID         object 
 1   RowID_propertyCode_bfeCode              int64  
 2   RowID_municipalityCode                  int64  
 3   RowID_propertyCode_esrCode              int64  
 4   Address_info_addressString              object 
 5   Address_info_streetString               object 
 6   Address_info_houseNumber                object 
 7   Address_info_floorString                object 
 8   Address_info_sideDoorNumber             object 
 9   Address_info_zipString_code_and_name    object 
 10  Address_info_zipCode                    int64  
 11  Feature_dummy_isApartment               bool   
 12  Feature_categorical_propertyTypeNumber  int64  
 13  Feature_Floor_level_of_property         object 
 14  Feature_category_propertyTypeName 

## ------ Save the result 

In [23]:
sub_columns = [
    'RowID_MAIN_boliga_ROW_ID_unitID', 

    'salesInfo_isSalesValid',   
    'salesInfo_handoverCode', 
    'salesInfo_handoverName', 
    'salesInfo_deedIssueDate', 
    'salesInfo_price',
    'salesInfo_recalculationDate', 
    'salesInfo_rebuildYear'
]

path=r'D:\Thesis\Properties\Denmark\RE_due_scraping_properties\Boliga_dk\Creating_main_dataset_for_sales_data\Data_split\11_salesInfos\Ready\Boliga_propertySales_salesInfos_Ready.csv'

data_salesInfo[sub_columns].to_csv(path, encoding='utf-8', index=False)

In [ ]:
data_additionarBuildingInfo.info()